In [1]:
import os
import PIL
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
os.getcwd()

'/home/saquib/Documents/LipNet/codes'

In [3]:
os.chdir('..')

In [4]:
os.chdir('Dataset')
os.chdir('dev')
train = []
label = []
for f in os.listdir():
    os.chdir(f)
    for p in os.listdir():
        img = plt.imread(p)
        label.append(f)
        train.append(img)
    os.chdir('..')
    os.chdir('..')
    os.chdir('Train')
    os.chdir(f)
    for p in os.listdir():
        img = plt.imread(p)
        label.append(f)
        train.append(img)
    os.chdir('..')
    os.chdir('..')
    os.chdir('dev')

In [5]:
train = np.array(train)
label = np.array(label)

In [6]:
from keras.models import Sequential # To initialise the nn as a sequence of layers
from keras.layers import Convolution2D # To make the convolution layer for 2D images
from keras.layers import MaxPooling2D # 
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import CSVLogger
from keras.optimizers import RMSprop
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

/home/saquib/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
csv=CSVLogger("train_dev_combine.log")
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32,(2,2),input_shape = (224,224,1),strides=2,name='convo1'))
classifier.add(Convolution2D(64,(3,3), activation = 'relu',name='convo2'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 1 - Convolution
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo3'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(GlobalAveragePooling2D())
# As our model is still facing the problem so, we need to increase the regulization
classifier.add(Dropout((0.5)))
classifier.add(Dense(20, activation = 'softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [22]:
classifier.compile(optimizer = Adam(epsilon = 0.01,lr = 0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
from sklearn.cross_validation import train_test_split

/home/saquib/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
from keras import backend as K

In [10]:
import keras 

In [12]:
train.shape

(2800, 224, 224)

In [13]:
label.shape

(2800,)

In [25]:
def start(i):
    X_train,X_test,y_train,y_test = train_test_split(train,label,test_size = 0.2)
    X_shape = X_train.shape
    print('Epoch-{}'.format(i))
    img_rows = 224
    img_cols = 224
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_rows, img_cols)
    else:
        input_shape = (img_rows, img_cols, 1)
    X_train_s = X_train.reshape((X_train.shape[0],) + input_shape)
    X_train_s = X_train_s.astype('float32')
    X_test_s = X_test.reshape((X_test.shape[0],) + input_shape)
    X_test_s = X_test_s.astype('float32')
    X_train_s /= 255
    X_test_s /= 255
    y_train = keras.utils.to_categorical(y_train, 20)
    y_test = keras.utils.to_categorical(y_test, 20)
    classifier.fit(X_train_s,y_train,batch_size=32,epochs=1,validation_data=(X_test_s, y_test))

In [21]:
for i in range(10):
    start(i)

Scaling the data
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 209s 93ms/step - loss: 2.9993 - acc: 0.0473 - val_loss: 2.9963 - val_acc: 0.0464
Scaling the data
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 208s 93ms/step - loss: 2.9974 - acc: 0.0482 - val_loss: 2.9975 - val_acc: 0.0357
Scaling the data
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 205s 92ms/step - loss: 2.9914 - acc: 0.0585 - val_loss: 2.9849 - val_acc: 0.0679
Scaling the data
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 206s 92ms/step - loss: 2.9685 - acc: 0.0674 - val_loss: 3.0047 - val_acc: 0.0661
Scaling the data
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 291s 130ms/step - loss: 2.9636 - acc: 0.0670 - val_loss: 2.9436 - val_acc: 0.0893

In [26]:
for i in range(10):
    start(i)

Epoch-0
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 209s 93ms/step - loss: 2.8915 - acc: 0.0830 - val_loss: 2.8603 - val_acc: 0.0964
Epoch-1
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 207s 92ms/step - loss: 2.8833 - acc: 0.0920 - val_loss: 2.8639 - val_acc: 0.1125
Epoch-2
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 208s 93ms/step - loss: 2.8841 - acc: 0.0879 - val_loss: 2.8497 - val_acc: 0.0839
Epoch-3
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 209s 93ms/step - loss: 2.8855 - acc: 0.0821 - val_loss: 2.8406 - val_acc: 0.1179
Epoch-4
Train on 2240 samples, validate on 560 samples
Epoch 1/1
2240/2240 [==============================] - 233s 104ms/step - loss: 2.8626 - acc: 0.1022 - val_loss: 2.8730 - val_acc: 0.0911
Epoch-5
Train on 2240 samples, validate on 5